# EDA and Cleaning

In [179]:
import requests
import time
import pandas as pd
import json
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
import matplotlib.pyplot as plt
import numpy as np
from xml.etree import ElementTree

In [77]:
%run ../bx_sql.py

## Configure PostgreSQL Server with Docker

In [79]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

## Retrieve Data from PostgreSQL Database

### Books

In [91]:
book_query = '''SELECT * FROM books;
'''

book_results = execute_query(book_query, dict_cur=True)

In [92]:
book_df = pd.DataFrame(book_results)

In [93]:
len(book_df)

271379

In [94]:
book_df.head()

,book_author,book_title,image_url_l,image_url_m,image_url_s,isbn,publisher,year_of_publication
0,Mark P. O. Morford,Classical Mythology,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0195153448,Oxford University Press,2002
1,Richard Bruce Wright,Clara Callan,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,0002005018,HarperFlamingo Canada,2001
2,Carlo D'Este,Decision in Normandy,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,0060973129,HarperPerennial,1991
3,Gina Bari Kolata,Flu: The Story of the Great Influenza Pandemic...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,0374157065,Farrar Straus Giroux,1999
4,E. J. W. Barber,The Mummies of Urumchi,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0393045218,W. W. Norton &amp; Company,1999


In [95]:
book_df.drop(columns=['image_url_l', 'image_url_m', 'image_url_s'], inplace=True)

In [96]:
book_df.head()

,book_author,book_title,isbn,publisher,year_of_publication
0,Mark P. O. Morford,Classical Mythology,0195153448,Oxford University Press,2002
1,Richard Bruce Wright,Clara Callan,0002005018,HarperFlamingo Canada,2001
2,Carlo D'Este,Decision in Normandy,0060973129,HarperPerennial,1991
3,Gina Bari Kolata,Flu: The Story of the Great Influenza Pandemic...,0374157065,Farrar Straus Giroux,1999
4,E. J. W. Barber,The Mummies of Urumchi,0393045218,W. W. Norton &amp; Company,1999


In [122]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
book_author            271379 non-null object
book_title             271379 non-null object
isbn                   271379 non-null object
publisher              271379 non-null object
year_of_publication    271379 non-null int64
dtypes: int64(1), object(4)
memory usage: 10.4+ MB


#### Missing Data

In [97]:
book_df.isna().sum()

book_author            1
book_title             0
isbn                   0
publisher              2
year_of_publication    0
dtype: int64

##### Author

In [113]:
book_df[book_df['book_author'].isna()]

,book_author,book_title,isbn,publisher,year_of_publication
187714,None,The Credit Suisse Guide to Managing Your Perso...,9627982032,Edinburgh Financial Publishing,1995


[Amazon (The Credit Suisse Guide to Managing Your Personal Wealth)](https://www.amazon.ca/Credit-Suisse-Managing-Personal-Wealth/dp/9627982075)

##### Publisher

In [117]:
book_df.loc[187714, ['book_author']] = 'Larissa Anne Downes'

In [100]:
book_df[book_df['publisher'].isna()]

,book_author,book_title,isbn,publisher,year_of_publication
128920,Elaine Corvidae,Tyrant Moon,193169656X,None,2002
129068,Linnea Sinclair,Finders Keepers,1931696993,None,2001


Sources:
-  [AbeBooks (Tyrant Moon)](https://www.abebooks.com/9781931696562/Tyrant-Moon-Elaine-Corvidae-193169656X/plp)
-  [Google Books (Finders Keepers)](https://books.google.com/books/about/Finders_Keepers.html?id=M9pbPgAACAAJ)

In [155]:
book_df.loc[128920, ['publisher']] = 'Novelbooks Incorporated'

In [110]:
book_df.loc[129068, ['publisher']] = 'Novelbooks Incorporated'

##### Year of Publication ????

In [162]:
np.asarray(sorted(book_df['year_of_publication'].unique()))

array([   0, 1376, 1378, 1806, 1897, 1900, 1901, 1902, 1904, 1906, 1908,
       1909, 1910, 1911, 1914, 1917, 1919, 1920, 1921, 1922, 1923, 1924,
       1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935,
       1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946,
       1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,
       1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,
       1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2008, 2010, 2011, 2012, 2020, 2021,
       2024, 2026, 2030, 2037, 2038, 2050])

In [171]:
book_df[book_df['year_of_publication'] == 0].head()

,book_author,book_title,isbn,publisher,year_of_publication
177,Schiller,Kabale Und Liebe,3150000335,"Philipp Reclam, Jun Verlag GmbH",0
189,Gabriel Garcia Marquez,Die Liebe in Den Zelten,342311360X,Deutscher Taschenbuch Verlag (DTV),0
289,Barbara Kingsolver,Poisonwood Bible Edition Uk,0571197639,Faber Faber Inc,0
352,Golding,"Herr Der Fliegen (Fiction, Poetry and Drama)",3596214629,Fischer Taschenbuch Verlag GmbH,0
542,P Coelho,Biblioteca Universale Rizzoli: Sulla Sponda De...,8845229041,Fabbri - RCS Libri,0


In [169]:
book_df[book_df['year_of_publication'] >= 2005].head()

,book_author,book_title,isbn,publisher,year_of_publication
3547,Grant Naylor,Red Dwarf,0140124373,Penguin Books Ltd,2005
12799,Kathleen E. Woodiwiss,A Rose in Winter,0380816792,Harper Mass Market Paperbacks,2011
26187,Frances Sakoian,The Astrologer's Handbook,006272004X,HarperResource,2005
30030,Kitty Kelley,The Royals,068160204X,Bausch &amp; Lombard,2020
37505,Coville,MY TEACHER FRIED MY BRAINS (RACK SIZE) (MY TEA...,0671746103,Aladdin,2030


In [181]:
# url = 'https://www.goodreads.com/book/isbn/ISBN?format=xml&isbn=3150000335&key=prkLqGzgF7KgNNy3VqgJpQ'

##### ISBN

In [128]:
book_df['isbn'].nunique()

271379

##### Title

In [132]:
book_df['book_title'].nunique()

242154

In [145]:
book_df[book_df['book_title'].duplicated()].head()

,book_author,book_title,isbn,publisher,year_of_publication
240,Dan Brown,Angels &amp; Demons,0743486226,Atria,2003
356,Roald Dahl,James and the Giant Peach,0140374248,Penguin USA (Paper),2000
406,Sebastian Junger,The Perfect Storm : A True Story of Men Agains...,006101351X,HarperTorch,1998
457,John Grisham,The Firm,044021145X,Bantam Dell Publishing Group,1992
476,T. Coraghessan Boyle,The Tortilla Curtain,014023828X,Penguin Books,1996


In [136]:
book_df[book_df['book_title'] == 'Angels &amp; Demons']

,book_author,book_title,isbn,publisher,year_of_publication
119,Dan Brown,Angels &amp; Demons,0671027360,Pocket Star,2001
240,Dan Brown,Angels &amp; Demons,0743486226,Atria,2003
28021,Dan Brown,Angels &amp; Demons,0671027352,Atria,2000
36952,Dan Brown,Angels &amp; Demons,0743535774,Audioworks,2003
163571,Dan Brown,Angels &amp; Demons,0743535766,Simon &amp; Schuster Audio,2003


**Merge with users_df and change `isbn` to only one**

### Users

In [224]:
user_query = '''SELECT * FROM users;
'''

user_results = execute_query(user_query, dict_cur=True)

In [225]:
users_df = pd.DataFrame(user_results)

In [226]:
users_df.head()

,age,location,user_id
0,None,"nyc, new york, usa",1
1,18.0,"stockton, california, usa",2
2,None,"moscow, yukon territory, russia",3
3,17.0,"porto, v.n.gaia, portugal",4
4,None,"farnborough, hants, united kingdom",5


In [227]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
age         168096 non-null object
location    278858 non-null object
user_id     278858 non-null int64
dtypes: int64(1), object(2)
memory usage: 6.4+ MB


In [233]:
users_df['user_id'].nunique()

278858

In [257]:
users_df['city'] = users_df['location'].apply(lambda x: x.split(',')[0])

In [261]:
for i in users_df['location']:
    if ',' not in i:
        print(i)

lawrenceville


In [262]:
users_df[users_df['location'] == 'lawrenceville']

,age,location,user_id,city,state,country
134376,30.0,lawrenceville,134377,lawrenceville,NaN,italy


In [264]:
users_df.loc[134376, 'location'] = 'lawrenceville, , '

In [265]:
users_df['state'] = users_df['location'].apply(lambda x: x.split(',')[1])

In [266]:
users_df['country'] = users_df['location'].apply(lambda x: x.split(',')[2])

IndexError: list index out of range